<a href="https://colab.research.google.com/github/PasinduWaidyarathna/Deep-Learning-Mini-Project-03/blob/main/test03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary Library Imports

In [1]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

Mount the Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read the data file

In [3]:
text_file = "/content/drive/MyDrive/spa.txt"
with open(text_file) as f:
   lines = f.read().split("\n")[:-1]

i = 0
for line in lines:
   print(line)
   i = i + 1
   if(i==20):
     break



Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.
Run!	¡Corre!
Run.	Corred.
Who?	¿Quién?
Fire!	¡Fuego!
Fire!	¡Incendio!
Fire!	¡Disparad!
Help!	¡Ayuda!
Help!	¡Socorro! ¡Auxilio!
Help!	¡Auxilio!
Jump!	¡Salta!
Jump.	Salte.
Stop!	¡Parad!
Stop!	¡Para!
Stop!	¡Pare!
Wait!	¡Espera!


In [4]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

You can't view Flash content on an iPad. However, you can easily email yourself the URLs of these web pages and view that content on your regular computer when you get home.	No puedes ver contenido en Flash en un iPad. Sin embargo, puedes fácilmente enviarte por correo electrónico las URL's de esas páginas web y ver el contenido en tu computadora cuando llegas a casa.
A mistake young people often make is to start learning too many languages at the same time, as they underestimate the difficulties and overestimate their own ability to learn them.	Un error que cometen a menudo los jóvenes es el de comenzar a aprender demasiadas lenguas al mismo tiempo, porque subestiman sus dificultades y sobrestiman sus propias capacidades para aprenderlas.
No matter how much you try to convince people that chocolate is vanilla, it'll still be chocolate, even though you may manage to convince yourself and a few others that it's vanilla.	No importa cuánto insistas en convencer a la gente de que el chocol

Split the English and Spanish translation pairs

In [5]:
text_pairs = []
for line in lines:
   english, spanish = line.split("\t")
   spanish = "[start] " + spanish + " [end]"
   text_pairs.append((english, spanish))


for i in range(3):
  print(random.choice(text_pairs))


('He has more money than is needed.', '[start] Él tiene más dinero de lo que se necesita. [end]')
('When does your trip begin?', '[start] ¿Cuándo empieza tu viaje? [end]')
('Married people are happier than unmarried people.', '[start] La gente casada es más feliz que la gente soltera. [end]')


Randomize the data

In [6]:
random.shuffle(text_pairs)

Spliting the data into training, validation and Testing

In [7]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))


Total sentences: 118964
Training set size: 83276
Validation set size: 17844
Testing set size: 17844


In [8]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

118964

Removing Punctuations

In [9]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [10]:
f"{3+5}"

'8'

Vectorizing the English and Spanish text pairs

In [11]:
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
      lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)


Preparing datasets for the translation task

In [12]:
batch_size = 64

def format_dataset(eng, spa):
   eng = source_vectorization(eng)
   spa = target_vectorization(spa)
   return ({
         "english": eng,
         "spanish": spa[:, :-1],
   }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


for inputs, targets in train_ds.take(1):
   print(f"inputs['english'].shape: {inputs['english'].shape}")
   print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
   print(f"targets.shape: {targets.shape}")




inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [13]:
print(list(train_ds.as_numpy_iterator())[50])

({'english': array([[  82,  211,    6, ...,    0,    0,    0],
       [   2, 8526, 4518, ...,    0,    0,    0],
       [ 692,   41,   15, ...,    0,    0,    0],
       ...,
       [   3,   16,    4, ...,    0,    0,    0],
       [  15,    5,  115, ...,    0,    0,    0],
       [  45, 2170,   20, ...,    0,    0,    0]]), 'spanish': array([[   2,   81, 1977, ...,    0,    0,    0],
       [   2,   32, 6300, ...,    0,    0,    0],
       [   2,   42,   17, ...,    0,    0,    0],
       ...,
       [   2,   46,    5, ...,    0,    0,    0],
       [   2,    4,  108, ...,    0,    0,    0],
       [   2,   75, 8259, ...,    0,    0,    0]])}, array([[   81,  1977,     6, ...,     0,     0,     0],
       [   32,  6300, 13872, ...,     0,     0,     0],
       [   42,    17,    71, ...,     0,     0,     0],
       ...,
       [   46,     5,    53, ...,     0,     0,     0],
       [    4,   108,   342, ...,     0,     0,     0],
       [   75,  8259,  2315, ...,     0,     0,     0]]

Transformer encoder implemented as a subclassed Layer

In [14]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
      super().__init__(**kwargs)
      self.embed_dim = embed_dim
      self.dense_dim = dense_dim
      self.num_heads = num_heads
      self.attention = layers.MultiHeadAttention(
           num_heads=num_heads, key_dim=embed_dim)
      self.dense_proj = keras.Sequential(
           [layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),]
      )
      self.layernorm_1 = layers.LayerNormalization()
      self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
      if mask is not None:
         mask = mask[:, tf.newaxis, :]
      attention_output = self.attention(
         inputs, inputs, attention_mask=mask)
      proj_input = self.layernorm_1(inputs + attention_output)
      proj_output = self.dense_proj(proj_input)
      return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


The Transformer decoder

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
      super().__init__(**kwargs)
      self.embed_dim = embed_dim
      self.dense_dim = dense_dim
      self.num_heads = num_heads
      self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
      self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
      self.dense_proj = keras.Sequential(
          [layers.Dense(dense_dim, activation="relu"),
           layers.Dense(embed_dim),]
      )
      self.layernorm_1 = layers.LayerNormalization()
      self.layernorm_2 = layers.LayerNormalization()
      self.layernorm_3 = layers.LayerNormalization()
      self.supports_masking = True

    def get_config(self):
      config = super().get_config()
      config.update({
          "embed_dim": self.embed_dim,
          "num_heads": self.num_heads,
          "dense_dim": self.dense_dim,
      })
      return config

    def get_causal_attention_mask(self, inputs):
      input_shape = tf.shape(inputs)
      batch_size, sequence_length = input_shape[0], input_shape[1]
      i = tf.range(sequence_length)[:, tf.newaxis]
      j = tf.range(sequence_length)
      mask = tf.cast(i >= j, dtype="int32")
      mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
      mult = tf.concat(
              [tf.expand_dims(batch_size, -1),
               tf.constant([1, 1], dtype=tf.int32)], axis=0)
      return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
      causal_mask = self.get_causal_attention_mask(inputs)
      if mask is not None:
           padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
           padding_mask = tf.minimum(padding_mask, causal_mask)
      else:
           padding_mask = mask
      attention_output_1 = self.attention_1(
           query=inputs,
           value=inputs,
           key=inputs,
           attention_mask=causal_mask)
      attention_output_1 = self.layernorm_1(inputs + attention_output_1)
      attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
       )
      attention_output_2 = self.layernorm_2(
          attention_output_1 + attention_output_2)
      proj_output = self.dense_proj(attention_output_2)
      return self.layernorm_3(attention_output_2 + proj_output)



Positional Encoding

In [16]:
class PositionalEmbedding(layers.Layer):
     def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
         super().__init__(**kwargs)
         self.token_embeddings = layers.Embedding(
             input_dim=input_dim, output_dim=output_dim)
         self.position_embeddings = layers.Embedding(
             input_dim=sequence_length, output_dim=output_dim)
         self.sequence_length = sequence_length
         self.input_dim = input_dim
         self.output_dim = output_dim

     def call(self, inputs):
         length = tf.shape(inputs)[-1]
         positions = tf.range(start=0, limit=length, delta=1)
         embedded_tokens = self.token_embeddings(inputs)
         embedded_positions = self.position_embeddings(positions)
         return embedded_tokens + embedded_positions

     def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

     def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
             "output_dim": self.output_dim,
             "sequence_length": self.sequence_length,
             "input_dim": self.input_dim,
        })
        return config


End-to-end Transformer

In [17]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [18]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 spanish (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['spanish[0][0]']         

Training the sequence-to-sequence Transformer

In [19]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
1302/1302 [==============================] - 109s 76ms/step - loss: 3.7981 - accuracy: 0.4396 - val_loss: 2.8855 - val_accuracy: 0.5337
Epoch 2/30
1302/1302 [==============================] - 90s 69ms/step - loss: 2.8583 - accuracy: 0.5486 - val_loss: 2.5145 - val_accuracy: 0.5891
Epoch 3/30
1302/1302 [==============================] - 91s 70ms/step - loss: 2.5612 - accuracy: 0.5928 - val_loss: 2.3841 - val_accuracy: 0.6131
Epoch 4/30
1302/1302 [==============================] - 91s 70ms/step - loss: 2.3982 - accuracy: 0.6188 - val_loss: 2.3110 - val_accuracy: 0.6266
Epoch 5/30
1302/1302 [==============================] - 91s 70ms/step - loss: 2.2944 - accuracy: 0.6371 - val_loss: 2.2977 - val_accuracy: 0.6310
Epoch 6/30
1302/1302 [==============================] - 91s 70ms/step - loss: 2.2184 - accuracy: 0.6513 - val_loss: 2.2828 - val_accuracy: 0.6397
Epoch 7/30
1302/1302 [==============================] - 91s 70ms/step - loss: 2.1597 - accuracy: 0.6628 - val_loss: 2.2514 

Translating new sentences with our Transformer model

In [20]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


Output Testing and Decoding the output sequence

In [21]:
import numpy as np

def decode_sequence(input_sentence):
     tokenized_input_sentence = source_vectorization([input_sentence])
     decoded_sentence = "[start]"
     for i in range(max_decoded_sentence_length):
       tokenized_target_sentence = target_vectorization(
         [decoded_sentence])[:, :-1]
       predictions = transformer(
         [tokenized_input_sentence, tokenized_target_sentence])
       sampled_token_index = np.argmax(predictions[0, i, :])
       sampled_token = spa_index_lookup[sampled_token_index]
       decoded_sentence += " " + sampled_token
       if sampled_token == "[end]": break
     return decoded_sentence

Transformer translating output

In [22]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))



-
Physics is a branch of science.
[start] de física es un arma de la ciencia [end]
-
I lost my watch.
[start] perdí a mi reloj [end]
-
He often accepted bad advice.
[start] Él a menudo aceptó su novia [end]
-
I don't know how to interpret his words.
[start] no sé cómo se [UNK] sus palabras [end]
-
We should remain quiet.
[start] deberíamos seguir hacia silencio [end]


Evaluation using the BLEU score

In [23]:
from nltk.translate.bleu_score import sentence_bleu

test_eng_texts = [pair[0] for pair in test_pairs]
test_spa_texts = [pair[1] for pair in test_pairs]
score = 0
bleu  = 0
for i in range(20):
    candidate = decode_sequence(test_eng_texts[i])
    reference = test_spa_texts[i].lower()
    print(candidate,reference)
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    bleu+=score
    print(f"Score:{score}")
print(f"\nBLEU score : {round(bleu,2)}/20")

[start] qué tiene que hacer conmigo [end] [start] ¿qué tiene que ver eso conmigo? [end]
Score:0.43902439024390244


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

[start] los lunes no son buenos [end] [start] el lunes no va bien. [end]
Score:0.3783783783783784
[start] tom es una [UNK] [end] [start] tom es torpe. [end]
Score:0.4
[start] no puedes [UNK] [end] [start] no puedes perdértelo. [end]
Score:0.4482758620689655
[start] tiene que estar borracho demasiado anoche [end] [start] él debe haber bebido demasiado anoche. [end]
Score:0.2909090909090909
[start] tom nunca come [UNK] [end] [start] tom nunca come quiché. [end]
Score:0.4117647058823529
[start] me [UNK] el sombrero que le [UNK] por el viento [end] [start] el viento se llevó mi sombrero. [end]
Score:0.26229508196721313
[start] sabes que algo no sé [end] [start] ¿sabes tú algo que yo no sepa? [end]
Score:0.47058823529411764
[start] crees en los [UNK] [end] [start] ¿crees en los ángeles? [end]
Score:0.40625
[start] me [UNK] [end] [start] yo cuidaré a los caballos. [end]
Score:0.45454545454545453
[start] [UNK] por ahí adentro [end] [start] solo espera ahí. [end]
Score:0.4
[start] quieres toca

20 new sentences accuracy

In [27]:
manualTest = [
   ("I love learning new languages.", "Me encanta aprender nuevos idiomas."),
    ("This is a beautiful day.", "Hoy es un día hermoso."),
    ("She sings very well.", "Ella canta muy bien."),
    ("They are going to the park.", "Van al parque."),
    ("The book is on the table.", "El libro está en la mesa."),
    ("We need to finish this project.", "Necesitamos terminar este proyecto."),
    ("He is a good friend.", "Él es un buen amigo."),
    ("I want to travel around the world.", "Quiero viajar por todo el mundo."),
    ("The cat is sleeping on the sofa.", "El gato está durmiendo en el sofá."),
    ("She cooks delicious food.", "Ella cocina comida deliciosa."),
    ("We have a lot of work to do.", "Tenemos mucho trabajo que hacer."),
    ("He speaks Spanish fluently.", "Él habla español con fluidez."),
    ("The concert starts at 7 PM.", "El concierto comienza a las 7 PM."),
    ("I need to buy some groceries.", "Necesito comprar algunas provisiones."),
    ("She is studying for her exam.", "Ella está estudiando para su examen."),
    ("They are watching a movie.", "Están viendo una película."),
    ("He plays the guitar very well.", "Él toca la guitarra muy bien."),
    ("We are going to the beach tomorrow.", "Vamos a la playa mañana."),
    ("The museum is closed on Mondays.", "El museo está cerrado los lunes."),
    ("I am going to visit my family.", "Voy a visitar a mi familia.")
]

from nltk.translate.bleu_score import sentence_bleu

testENTexts = [pair[0] for pair in manualTest]
testSPTexts = [pair[1] for pair in manualTest]
score = 0
bleu  = 0
for i in range(20):
    candidate = decode_sequence(testENTexts[i])
    reference = testSPTexts[i].lower()
    print(candidate,reference)
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    bleu+=score
    print(f"Score:{score}")
print(f"\nBLEU score : {round(bleu,2)}/20")


[start] me encanta aprender nuevo lengua [end] me encanta aprender nuevos idiomas.
Score:0.3043478260869566
[start] es un día hermosa [end] hoy es un día hermoso.
Score:0.3870967741935484
[start] ella canta muy bien [end] ella canta muy bien.
Score:0.36363636363636365
[start] ellos van al parque [end] van al parque.
Score:0.30303030303030304
[start] el libro está sobre la mesa [end] el libro está en la mesa.
Score:0.3170731707317073
[start] necesitamos que [UNK] este proyecto [end] necesitamos terminar este proyecto.
Score:0.2653061224489796
[start] Él es un buen amigo [end] él es un buen amigo.
Score:0.36363636363636365
[start] quiero andar por toda la hora [end] quiero viajar por todo el mundo.
Score:0.3023255813953489
[start] el gato está durmiendo en el sofá [end] el gato está durmiendo en el sofá.
Score:0.34042553191489366
[start] ella cocina comida de arroz [end] ella cocina comida deliciosa.
Score:0.2682926829268293
[start] tenemos un montón de trabajo por hacer [end] tenemos mu